## 실시간 웹캠 -> 상자 검출 및 겹침 확인 해서 경고 까지

### 필요 라이브러리 불러오기

In [1]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox

### 상자 서로 겹치는 지 확인해주는 함수 : CheckOverlabBoxNum()에서 사용

In [8]:
def CheckOverlapBox(bbox1, bbox2): 
    check = 0
    #bbox 형식: [Bbox 왼쪽위(최소)x, Bbox 왼쪽위(최소)y, Bbox 오른쪽아래(최대), Bbox 오른쪽 아래(최대)y] 
    #b_x 형식 : [x좌표 최소, x좌표 최대]
    #b_y 형식 : [y좌표 최소, y좌표 최대]
    b1_x = [bbox1[0], bbox1[2]]
    b1_y = [bbox1[1], bbox1[3]]
    b2_x = [bbox2[0], bbox2[2]]
    b2_y = [bbox2[1], bbox2[3]]

    if b1_x[0]<=b2_x[1] and b1_x[1]>=b2_x[0]: # x축 방향에서 서로 중복됨
        if b1_y[0]<=b2_y[1] and b1_y[1]>=b2_y[0]: # y축 방향에서 서로 중복됨
            check = 1 # 겹침 
    
    return check # 두 박스가 겹치면 1, 안겹치면 0을 반환 


### 상자 최대 몇 개 겹치는 지 확인해주는 함수 : main run에서 사용

In [3]:
def CheckOverlapBoxNum(bbox):
    
    OverlapRelation = [0 for i in range(len(bbox))] # 겹침관계표현list 
    OverlapRelation[0] = 1
    maxnum = 1
    for i in range(len(bbox)):
        if OverlapRelation[i] == 0:
            OverlapRelation[i] = ++maxnum
        for j in range(0, i):
            if CheckOverlapBox(bbox[i], bbox[j]) == 1: #상자가 서로 겹친다면
                OverlapRelation[i] = OverlapRelation[j]
                break
                
                    
    MaxOverlapBoxNum = 1
    for i in range(1, max(OverlapRelation)+1):
        if MaxOverlapBoxNum < OverlapRelation.count(i):
            MaxOverlapBoxNum = OverlapRelation.count(i)
    
    #return MaxOverlapBoxNum
    return [OverlapRelation, MaxOverlapBoxNum]

* return 값 (최종 겹침 결과 배열) 설명 : 
 * [OverlapRelation, MaxOverlapBoxNum] : [겹침관계표현list, 최대상자겹침갯수]
 * 겸침관계표현list : 경계상자 수만큼의 크기를 가지는 리스트, 같은 숫자를 가지는 bbox는 같은 연결그룹에 속함
 * 최대상자겹침갯수 : 한번에 연결되어 있는 상자 그룹 중, 그룹에 속한 상자 수의 최댓값 

### main run : 웹캠 열어서 판별, 'q' 키 입력 시 종료 

In [7]:
# open webcam (웹캠 열기)
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
# loop through frames
while webcam.isOpened():
    
    # read frame from webcam 
    status, frame = webcam.read() 
    
    if not status: 
        break
        
    # apply object detection (물체 검출 -> 사람만)
    bbox, label, conf = cv.detect_common_objects(frame) # cvlib의 객체 검출 실행 
                                # -> 결과값 : bbox정보([x1,y1,x2,y2]), 분류 object label, 신회도 conf
    a_bbox = [] 
    a_label = [] 
    a_conf = [] 
    for i in range(len(bbox)): # common object에 대해 실행된 검출 결과에서 사람에 대한 결과만 추출
        if label[i] == 'person':
            a_bbox.append(bbox[i])
            a_label.append(label[i])
            a_conf.append(conf[i])

    # draw bounding box over detected objects (사람 경계상자 그리기)
    out = draw_bbox(frame, a_bbox, a_label, a_conf, write_conf=True)   
    print(a_bbox, a_label, a_conf)
    
    # display output (실시간 화면 출력)
    cv2.imshow("Real-time object detection", out)
            
    if len(bbox)>0: # 
        CheB = CheckOverlapBoxNum(bbox) # 경계상자 간 겹침정보 확인 
        print(CheB) 
        # 5인 이상 밀집 시 경고 메세지 발생 
        if CheB[1] >= 5 : # 한 번에 밀집된 사람수 (연결그룹에 속한 경계상자 수의 최댓값)이 5 이상이면 
            print("5인 이상 밀집 ! 거리두기를 해주세요 ")
    
    #---------------------------------------------------
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows()

[[87, 78, 653, 481]] ['person'] [0.8756798505783081]
[[1], 1]
[[104, 102, 648, 480]] ['person'] [0.7574600577354431]
[[1], 1]
[[108, 96, 638, 479]] ['person'] [0.7786049246788025]
[[1], 1]
[[114, 97, 634, 478]] ['person'] [0.7870376706123352]
[[1], 1]
[[92, 100, 656, 480]] ['person'] [0.7472249865531921]
[[1], 1]
[[102, 99, 646, 478]] ['person'] [0.7210478186607361]
[[1], 1]
[[106, 99, 645, 475]] ['person'] [0.773526668548584]
[[1], 1]
[[103, 96, 649, 476]] ['person'] [0.7707498669624329]
[[1], 1]
[[103, 99, 648, 474]] ['person'] [0.7554253935813904]
[[1], 1]
[[88, 97, 657, 476]] ['person'] [0.7689708471298218]
[[1], 1]
[[88, 92, 655, 479]] ['person'] [0.7713832259178162]
[[1], 1]
[[96, 93, 647, 476]] ['person'] [0.7202344536781311]
[[1], 1]
[[84, 98, 652, 478]] ['person'] [0.722913384437561]
[[1], 1]
